In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures


In [2]:
data = pd.read_excel("C:/Users/PCC/Downloads/Nifty50 Stock Price Prediction.xlsx")

In [3]:
data.head(1)
data.shape

(49690, 8)

In [4]:
data.dropna(inplace=True)

In [5]:
data['Date'] = pd.to_datetime(data['Date'])

In [8]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Initialize credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('key.json', scope)
client = gspread.authorize(creds)

# Open your Google Sheet by title or URL
sheet = client.open('new')

# Function to update data from the sheet
def update_data():
    
    # data = worksheet.get_all_values()
    # Access a specific worksheet within the Google Sheet
    worksheet = sheet.worksheet('Sheet1')  # Replace 'Worksheet Title' with your worksheet title

    # Fetch data from a specific column (e.g., column A)
    column_values = worksheet.col_values(6)  # Replace '1' with the column number you want

    # Initialize an empty list to store the values
    data_list = []

    # Append the column values to the list
    data_list.extend(column_values)
    # print(data_list)
    stock_name = data_list[1]
    qty = int(data_list[2])
    buying_date = data_list[3]
    selling_date = data_list[4]
    pnl = data_list[7]
    

    new_data = data[data['Stock Name']== stock_name]
    new_data = new_data.copy()
    new_data['Year'] = new_data['Date'].dt.year
    new_data['Month'] = new_data['Date'].dt.month
    new_data['Day'] = new_data['Date'].dt.day

    y = new_data['Close']

    x = new_data[['Volume', 'Chg%', 'Year', 'Month', 'Day']]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    
    # Create polynomial features
    poly = PolynomialFeatures(degree=2)
    x_train_poly = poly.fit_transform(x_train)
    x_test_poly = poly.transform(x_test)


    lr = LinearRegression()

    # Train the model
    lr.fit(x_train_poly, y_train)
    y_pred=lr.predict(x_test_poly)

    y_predt=lr.predict(x_train_poly)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = x_test.shape[1]
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)


    print("R-squared score:", r2)
    print("Adjusted R-squared score:", adj_r2)

    new_data['Date'] = new_data['Date'].astype(str)
    new_data['Date'] = new_data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())


    # Convert the buying and selling dates to datetime.date objects
    buying_date = datetime.datetime.strptime(buying_date, '%Y-%m-%d').date()
    selling_date = datetime.datetime.strptime(selling_date, '%Y-%m-%d').date()

    # Get the current date
    today_date = datetime.date.today()



    # Extract data for features
    last_100_data = new_data.iloc[-100:]
    volume_mean = np.mean(last_100_data['Volume'])
    chg_mean = np.mean(last_100_data['Chg%'])
    volume_std = np.std(last_100_data['Volume'])
    chg_std = np.std(last_100_data['Chg%'])

    buying_year = buying_date.year
    buying_month = buying_date.month
    buying_day = buying_date.day
    selling_year = selling_date.year
    selling_month = selling_date.month
    selling_day = selling_date.day

    # Create feature vectors for buying and selling dates
    X_buying = np.array([[volume_mean, chg_mean,buying_year,buying_month,buying_day ]])
    X_buying_scaled = (X_buying - [volume_mean, chg_mean, 0, 0, 0]) / [volume_std, chg_std, 1, 1, 1]
    predicted_close_buying = lr.predict(poly.transform(X_buying_scaled))[-1]

    X_selling = np.array([[volume_mean, chg_mean,selling_year,selling_month,selling_day ]])
    X_selling_scaled = (X_selling - [volume_mean, chg_mean, 0, 0, 0]) / [volume_std, chg_std, 1, 1, 1]
    predicted_close_selling = lr.predict(poly.transform(X_selling_scaled))[-1]

    # Calculate profit or loss
    buying_price = predicted_close_buying
    selling_price = predicted_close_selling
    pnl = (selling_price - buying_price) * qty

    # Display the results
    print('Predicted Buying price:', predicted_close_buying)
    print('Predicted Selling price:', predicted_close_selling)
    print('Profit/Loss:', pnl)


    gc = gspread.service_account(filename="V:\Masai School Class\Project 3\key.json")
    wks = gc.open('new').sheet1
    wks.update('F8',pnl)
    print()
    print()



# Update data periodically (e.g., every 30 minutes)
from apscheduler.schedulers.blocking import BlockingScheduler
scheduler = BlockingScheduler()
scheduler.add_job(update_data, 'interval', minutes=0.05)
scheduler.start()

# Keep the script running
try:
    while True:
        pass
except (KeyboardInterrupt, SystemExit):
    # Shut down the scheduler gracefully on keyboard interrupt or system exit
    scheduler.shutdown()

R-squared score: 0.4303424073463692
Adjusted R-squared score: 0.41558443862477257
Predicted Buying price: 731.6324998892014
Predicted Selling price: 760.1734345977893
Profit/Loss: 28.540934708587884


c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
Execution of job "update_data (trigger: interval[0:00:03], next run at: 2023-10-20 03:40:54 IST)" skipped: maximum number of running instances reached (1)
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\gspread\worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(




R-squared score: 0.4303424073463692
Adjusted R-squared score: 0.41558443862477257


c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


Predicted Buying price: 731.6324998892014
Predicted Selling price: 760.1734345977893
Profit/Loss: 28.540934708587884


...



R-squared score: 0.4303424073463692
Adjusted R-squared score: 0.41558443862477257
Predicted Buying price: 646.0944548067819
Predicted Selling price: 760.1734345977893
Profit/Loss: 228.15795958201488


c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
Execution of job "update_data (trigger: interval[0:00:03], next run at: 2023-10-20 03:41:54 IST)" skipped: maximum number of running instances reached (1)
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\gspread\worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(




R-squared score: 0.4303424073463692
Adjusted R-squared score: 0.41558443862477257
Predicted Buying price: 646.0944548067819
Predicted Selling price: 760.1734345977893
Profit/Loss: 228.15795958201488


c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
Execution of job "update_data (trigger: interval[0:00:03], next run at: 2023-10-20 03:42:00 IST)" skipped: maximum number of running instances reached (1)
c:\Users\Vishal Chauhan\AppData\Local\Programs\Python\Python310\lib\site-packages\gspread\worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


KeyboardInterrupt: 